In [1]:
!pip install tensorflow


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

from tensorflow.keras import models, layers, callbacks
import joblib

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [3]:
df_main = pd.read_csv(r"C:\Users\ASUS\Desktop\Phishing_URL_detection\dataset.zip")
df_safe = pd.read_csv(r"C:\Users\ASUS\Desktop\Phishing_URL_detection\safe_urls.csv")

df = pd.concat([df_main, df_safe], ignore_index=True)
df.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [4]:
# Convert text labels to numeric values
df['Label'] = df['Label'].map({'bad': 1, 'good': 0})

# Check conversion
df.head()


,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1.0
3,mail.printakid.com/www.online.americanexpress....,1.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,1.0


In [ ]:
import re
import numpy as np
from urllib.parse import urlparse

def extract_features(url):
    url = str(url)
    features = []
    
    # Basic structure
    features.append(len(url))                          # URL length
    features.append(url.count('@'))                    # '@' count
    features.append(url.count('-'))                    # '-' count
    features.append(url.count('.'))                    # '.' count
    features.append(url.count('/'))                    # '/' count
    
    # Check for https
    features.append(1 if 'https' in url else 0)
    
    # Check for suspicious keywords
    keywords = ['login', 'secure', 'account', 'bank', 'verify', 'update']
    features.append(sum([1 for k in keywords if k in url.lower()]))
    
    return np.array(features)


In [ ]:
# Apply the feature extraction function
X = np.array([extract_features(url) for url in df['URL']])
y = df['Label'].values

print("✅ Features shape:", X.shape)
print("✅ Labels shape:", y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

import joblib
joblib.dump(scaler, 'scaler.pkl')

print("✅ Data ready for training!")


In [ ]:
from tensorflow.keras import models, layers, callbacks

model = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32,
    callbacks=[early_stop]
)


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Model Accuracy: {acc*100:.2f}%")


In [ ]:
model.save('phishing_model.h5')
print("✅ Model saved successfully!")


In [ ]:

# run in the same notebook where `model` and `scaler` exist
import joblib
import os

# change these if your project folder is elsewhere
models_dir = r"C:\Users\ASUS\Desktop\Phishing_URL_detection\models"
os.makedirs(models_dir, exist_ok=True)

# save scaler (if you have `scaler`)
joblib.dump(scaler, os.path.join(models_dir, 'scaler.pkl'))

# save model (if you used Keras)
model.save(os.path.join(models_dir, 'phishing_model.h5'))   # or .keras if you prefer

print("Saved to:", models_dir)
print("Files now:", os.listdir(models_dir))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras import models, layers

import joblib

print("✅ All libraries imported!")
